In [79]:
import pandas as pd
import networkx as nx

# Load dataset
file = './data.xlsx'
data = pd.read_excel(file)

# Initialize graph
G = nx.Graph()


def generate(datafile, graph):
    for index, row in datafile.iterrows():  
        # Node will be named after 'city, state'
        node_name = f"{row['city']}, {row['state']}"
        # Parse lists
        connected_cities = row['connected cities'].split(',')
        connected_states = row['connected cities states'].split(',')
        distances = [int(dist) for dist in row['distance to city'].split(',')]

        # Add the current city if not already in graph or update with lat, long, & interstates
        graph.add_node(node_name, lat=row['lat'], long=row['long'], interstates=row['interstates'])

        # Add edges
        for i, (connected_city, connected_state) in enumerate(zip(connected_cities, connected_states)):
            connected_city_state = f"{connected_city.strip()}, {connected_state.strip()}"
            if connected_city_state not in graph:
                graph.add_node(connected_city_state)
            # Distance is cost
            graph.add_edge(node_name, connected_city_state, distance=distances[i])


generate(data, G)

nodes = G.number_of_nodes()
edges = G.number_of_edges()
print("nodes: ", nodes, " | edges: ", edges)
print(G)
print(G.edges("Birmingham, AL"))
#for line in nx.generate_adjlist(G):
    #print(line)

nodes:  318  | edges:  407
Graph with 318 nodes and 407 edges
[('Birmingham, AL', 'Anniston, AL'), ('Birmingham, AL', 'Decatur, AL'), ('Birmingham, AL', 'Gadsden, AL'), ('Birmingham, AL', 'Montgomery, AL'), ('Birmingham, AL', 'Tuscaloosa, AL')]


In [80]:
def bfs(start_city,end_city):
    counter = 0
    queue = []
    visited = []
    route = []
    previous_cities = []

    queue.append(start_city)
    
    while queue:
        current_city = queue.pop(0)
        visited.append(current_city)
        counter+=1

        if current_city == end_city:
            route.append(end_city)
            # route creating
            city = end_city
            for i in range(counter): #amount of times loopings needed to be done (not too sure if needed)
                for cities in previous_cities: #loops through list of city to check
                    if city == cities[0]: #matches the city to find the previous city
                        route.append(cities[1]) #adds previous city to the route
                        city = cities[1] #updates the city to backtrack
            route.reverse()
            print(route)
            print(counter)
            return route
        
        for connected_cities in G.edges(current_city):
            if connected_cities[1] not in visited:
                queue.append(connected_cities[1])
                previous_cities.append([connected_cities[1],current_city]) #to help backtrack later on

bfs('Anniston, AL', 'Auburn-Opelika, AL')

['Anniston, AL', 'Atlanta, GA', 'Auburn-Opelika, AL']
4


['Anniston, AL', 'Atlanta, GA', 'Auburn-Opelika, AL']